# Módulo 1 — Herramientas técnicas avanzadas (Python)

**Objetivo:** dominar la manipulación de series temporales financieras, regresiones avanzadas, y buenas prácticas reproducibles. Todos los comentarios en español.


## 1.1 — Setup y buenas prácticas

- Usar ambientes virtuales (conda/venv).
- Listado recomendado de librerías: `pandas`, `numpy`, `scipy`, `statsmodels`, `arch`, `matplotlib`, `seaborn` (opcional para exploración), `sklearn`.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
import statsmodels.api as sm

# mostrar versiones (útil para reproducibilidad)
print('numpy', np.__version__)
print('pandas', pd.__version__)


numpy 2.3.3
pandas 2.3.2


## 1.2 — Carga y limpieza de datos (esqueleto)

A continuación un esqueleto robusto para cargar series de precios, ajustar por splits/dividends y calcular retornos logarítmicos.


In [ ]:
def load_prices(path_or_ticker, source='csv'):
    """Carga precios desde un CSV local o (si se habilita) desde una API.
    Devuelve un DataFrame con índice datetime y columna 'close'."""
    if source=='csv':
        df = pd.read_csv(path_or_ticker, parse_dates=['date'], index_col='date')
        # mantener solo la columna close si existe
        if 'close' in df.columns:
            prices = df['close'].sort_index()
        else:
            # intentar columna alternativa
            prices = df.iloc[:,0].sort_index()
    else:
        raise NotImplementedError('Implementar API si necesario')
    # eliminar NaNs al inicio o final
    prices = prices.replace([np.inf, -np.inf], np.nan).dropna()
    return prices

# uso: prices = load_prices('path/a/archivo.csv')


## 1.3 — Cálculo de retornos y estadísticas básicas

Usamos retornos logarítmicos por propiedades aditivas en horizontes temporales.


In [ ]:
def compute_log_returns(prices):
    # prices: pd.Series con índice datetime
    logr = np.log(prices).diff().dropna()
    return logr


def summary_stats(returns):
    stats = {
        'mean': returns.mean(),
        'std': returns.std(),
        'skew': returns.skew(),
        'kurtosis': returns.kurtosis()
    }
    return pd.Series(stats)

# ejemplo (no ejecutado aquí):
# prices = pd.Series(...) ; r = compute_log_returns(prices) ; print(summary_stats(r))


## 1.4 — Regresión CAPM / Factor Models

Estimaremos beta y alpha con errores robustos y discutiremos interpretación.


In [ ]:
# Ejemplo de regresión CAPM con errores robustos
def capm_regression(returns_asset, returns_market, risk_free=0.0):
    """Calcula alpha y beta del activo respecto al mercado.
    returns_*: pd.Series de retornos en el mismo índice.
    risk_free: tasa continua (en el mismo periodo que returns, p.ej. diario).
    """
    # exceso de retorno
    y = returns_asset - risk_free
    X = returns_market - risk_free
    X = add_constant(X)
    model = OLS(y, X).fit(cov_type='HC3')  # errores robustos (HC3)
    return model

# uso: res = capm_regression(r_asset, r_market); print(res.summary())


## 1.5 — Backtesting reproducible (esqueleto)

Pautas para backtests: evitar look-ahead, aplicar slippage y comisiones, usar walk-forward.


In [ ]:
# Esqueleto simple de backtest (pseudocódigo reproducible)

def simple_backtest(signals, prices, freq='D', entry_horizon=1, slippage=0.0005, commission=0.0005):
    """signals: pd.Series con índice datetime y valores en {-1,0,1}
    prices: pd.Series de precios (close)
    Devuelve series de PnL y métricas básicas.
    """
    # alinear índices
    signals = signals.reindex(prices.index).fillna(0)
    # entradas en t aplicadas sobre precio en t+entry_horizon
    shifted_prices = prices.shift(-entry_horizon)
    # retornos de la estrategia: posición * (next_price/current_price - 1) - costs
    ret = signals * (shifted_prices / prices - 1)
    # aplicar slippage y comisión aproximada por transacción
    # este es un esqueleto; en implementaciones reales contabilizar volumen y tamaño de orden
    costs = (signals.diff().abs() > 0).astype(int) * (slippage + commission)
    strat_ret = ret - costs
    strat_ret = strat_ret.dropna()
    return strat_ret

# uso: pnl = simple_backtest(signals, prices)


## 1.6 — Buenas prácticas y reproducción
- Guardar versiones de datos (hash o snapshot).
- Registrar seeds en experimentos aleatorios.
- Documentar decisiones (README en cada carpeta de módulo).

---
